In [1]:
import pandas as pd
import numpy as np
from scipy.fftpack import fft
from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import ShuffleSplit

import matplotlib.pyplot as plt
from matplotlib.patches import Patch

In [2]:
rng = np.random.RandomState(1338)
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
cmap_group = plt.cm.Paired
cmap_y = plt.cm.coolwarm

def visualize_groups(classes, groups):
    # Visualize dataset groups
    fig, ax = plt.subplots(dpi=200) 
    ax.scatter(
        range(len(groups)),
        [0.5] * len(groups),
        c=groups,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.scatter(
        range(len(groups)),
        [3.5] * len(groups),
        c=classes,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.set(
        ylim=[-1, 5],
        yticks=[0.5, 3.5],
        yticklabels=["Data\ngroup", "Data\nclass"],
        xlabel="Sample index",
    )


def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data 
    )

    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data 
    )



    # Formatting
    yticklabels = list(range(n_splits)) + ["class", "group"]
    ax.set(
        yticks=np.arange(n_splits + 2) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 2.2, -0.2],
        xlim=[0, len(X)],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax

In [12]:
def create_segments(data, target_x, target_y, target_group,  window_size=80, step_size=40):
    segments = []
    labels = []
    group = []

    for i in range(0, len(data) - window_size, step_size):
        segments_data = []
        for x in target_x:
            x_values = data[x].values[i:i + window_size]
            segments_data.append(x_values)
        label = data[target_y].values[i]
        member = data[target_group].values[i]

        segments.append(segments_data)
        labels.append(label)
        group.append(member)
    return segments, labels, group


def create_label_segments(data, target, window_size=80, step_size=40):
    ys = []
    for i in range(0, len(data) - window_size, step_size):
        segments_data = []
        y = data[target].values[i]
        y.append(y)
    return ys


def extract_time_features(segments):
    features = []
    for segment in segments:
        segment_features = []
        for axis in segment:
            mean_axis = np.mean(axis)
            std_axis = np.std(axis)
            segment_features.extend([mean_axis, std_axis])
        features.append(segment_features)
    return np.array(features)

def extract_frequency_features(segments, sampling_rate=100):
    features = []
    for segment in segments:
        segment_features = []
        for axis in segment:
            fft_axis = np.abs(fft(axis))

            # Peak frequency and maximum amplitude
            peak_indices, _ = find_peaks(fft_axis)

            if len(peak_indices) > 0:
                peak_freq = peak_indices[np.argmax(fft_axis[peak_indices])] / len(axis) * sampling_rate
                max_amplitude = np.max(fft_axis[peak_indices])
            else:
                peak_freq = 0.0  # デフォルトの値を設定
                max_amplitude = 0.0  # デフォルトの値を設定

            # Signal energy
            energy = np.sum(axis ** 2) / len(axis)

            segment_features.extend([peak_freq, max_amplitude, energy])

        features.append(segment_features)

    return np.array(features)

In [59]:
# トイデータの読み込み
data = pd.read_csv('01_waistbelt_toy_dataset.csv', index_col=0)
# 欠損値の除去
data.dropna(axis=0, how="any", inplace=True)




In [66]:
# セグメントを用意し、時間領域の特徴量と周波数領域の特徴量を抽出
window_size = 80
step_size = 40
target_x = ['Accs.X', 'Accs.Y', 'Accs.Z']
target_y = "action" 
terget_group = "user_id"
target_names = list(data[target_y].unique())

# ラベルエンコーダーのインスタンス化
le0 = LabelEncoder()
data["action"] = le0.fit_transform(data["action"])

le1 = LabelEncoder()
data["user_id"] = le1.fit_transform(data["user_id"])

segments, labels, group = create_segments(data, target_x, target_y, terget_group, window_size, step_size)

time_features = extract_time_features(segments)
frequency_features = extract_frequency_features(segments)

# 時間領域の特徴量と周波数領域の特徴量を統合
combined_features = np.concatenate((time_features, frequency_features), axis=1)

# データセットの分割
X = combined_features
y = np.asarray(labels)

In [67]:
target_names

['jump',
 'lie',
 'run',
 'sit',
 'stand',
 'walk',
 'walk-climbdown',
 'walk-climbup']

In [68]:
# ランダムフォレスト分類器をインスタンス化
clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [71]:
# StratifiedKFoldのインスタンスを作成
skf = StratifiedKFold(n_splits=5)

all_y_true = []
all_y_pred = []

# 分割数、ここでは5分割
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    

    # モデルの学習
    clf.fit(X_train, y_train)
    
    # 予測
    y_pred = clf.predict(X_test)

    # 保存
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

all_y_true = le0.inverse_transform(np.array(all_y_true).astype(np.int32))
all_y_pred = le0.inverse_transform(np.array(all_y_pred).astype(np.int32))


In [72]:
# 全ユーザの結果に基づく混同行列と分類レポートを表示
print("Confusion Matrix:")
print(confusion_matrix(all_y_true, all_y_pred, labels=target_names))

print("\nClassification Report:")
print(classification_report(all_y_true, all_y_pred, labels=target_names))

Confusion Matrix:
[[149   0   0   0   0   1   0   0]
 [  0 149   0   0   0   0   1   0]
 [  0   0 147   0   0   0   1   2]
 [  0   0   3 147   0   0   0   0]
 [  0   0   0   9 140   1   0   0]
 [  0   0   1   0   0 140   8   1]
 [  0   0   2   0   0  10 129   9]
 [  0   0   1   0   0   3   6 138]]

Classification Report:
                precision    recall  f1-score   support

          jump       1.00      0.99      1.00       150
           lie       1.00      0.99      1.00       150
           run       0.95      0.98      0.97       150
           sit       0.94      0.98      0.96       150
         stand       1.00      0.93      0.97       150
          walk       0.90      0.93      0.92       150
walk-climbdown       0.89      0.86      0.87       150
  walk-climbup       0.92      0.93      0.93       148

      accuracy                           0.95      1198
     macro avg       0.95      0.95      0.95      1198
  weighted avg       0.95      0.95      0.95      1198

